In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
from numpy import zeros, hanning, hamming, ones, arange, fft, transpose, bartlett, blackman, kaiser
from scipy.signal import flattop, tukey, welch
from IPython.display import display
%matplotlib widget

In [ ]:
texts = [[], []]

with open("data.txt", "r") as file:
  textfile = file.readlines()

for text in textfile:
    text = text.split('\t')
    text[1] = text[1].replace('\n',"")
    texts[0].append(float(text[0]))
    texts[1].append(float(text[1]))


fig = plt.figure()
ax0 = fig.add_axes([0.1,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="ElectroCardioGram Signal")
ax0.plot(texts[0], texts[1])
plt.show()

In [ ]:
def shorttimefouriertransform(var, vartype, typelength, overlaplength):
    result = []
    windowedvar = zeros(len(var))
    windowfunction = windowtypefunction(vartype, typelength)
    
    for i in arange(0, len(var), (typelength-overlaplength)):
        for j in range(typelength):
            if i + j >= len(var):
                break
            windowedvar[i+j] = var[i+j] * windowfunction[j]
        result.append(abs(fft.fft(windowedvar)))
        windowedvar = zeros(len(var))

    return transpose(result)

In [ ]:
def windowtypefunction(vartype, typelength):
    if vartype == "Rectangular/None":
        return ones(typelength)
    elif vartype == "Hanning":
        return hanning(typelength)
    elif vartype == "Hamming":
        return hamming(typelength)
    elif vartype == "Bartlett":
        return bartlett(typelength)
    elif vartype == "Blackman":
        return blackman(typelength)
    elif vartype == "Flat Top":
        return flattop(typelength)
    elif vartype == "Tukey":
        return tukey(typelength)

In [ ]:
selectwindowtype = widgets.Select(
    options=['Bartlett', 'Blackman', 'Flat Top', 'Hanning', 'Hamming', 'Rectangular/None', 'Tukey'],
    value='Bartlett',
    description='Window Type:',
    disabled=False
)

selectwindowlength = widgets.IntText(
    value=50,
    description='Window Length:',
    disabled=False
)

selectoverlaplength = widgets.IntText(
    value=25,
    description='Overlap Window Length:',
    disabled=False
)

out = widgets.Output()
fig = plt.figure()
windowfunction = windowtypefunction(selectwindowtype.get_interact_value(), selectwindowlength.get_interact_value())
ax0 = fig.add_axes([0.1,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Window Signal")
ax0.plot(arange(len(windowfunction)), windowfunction)
plt.show()

def on_change(change):
    with out:
        # if change['type'] == 'change' and change['name'] == 'value':
        windowfunction = windowtypefunction(selectwindowtype.get_interact_value(), selectwindowlength.get_interact_value())
        fig.clear()
        ax0 = fig.add_axes([0.1,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Window Signal")
        ax0.plot(arange(len(windowfunction)), windowfunction)

selectwindowtype.observe(on_change)
selectwindowlength.observe(on_change)
display(selectwindowtype, selectwindowlength, selectoverlaplength, out)

In [ ]:
result = shorttimefouriertransform(texts[1], selectwindowtype.get_interact_value(), selectwindowlength.get_interact_value(), selectoverlaplength.get_interact_value())
fig1 = plt.figure()
im1 = plt.imshow(result, aspect='auto', extent=[0, 2, 0, 2000])
fig1.colorbar(im1)
plt.show()